In [1]:
import os

In [11]:
path1="D:\DS\Projects\project 2 - phonepe\pulse\data\aggregated\transaction\country\india\state"
agg_state_list=os.listdir(path1)

OSError: [WinError 123] The filename, directory name, or volume label syntax is incorrect: 'D:\\DS\\Projects\\project 2 - phonepe\\pulse\\data\x07ggregated\transaction\\country\\india\\state'

In [9]:
agg_state_list

['andaman-&-nicobar-islands',
 'andhra-pradesh',
 'arunachal-pradesh',
 'assam',
 'bihar',
 'chandigarh',
 'chhattisgarh',
 'dadra-&-nagar-haveli-&-daman-&-diu',
 'delhi',
 'goa',
 'gujarat',
 'haryana',
 'himachal-pradesh',
 'jammu-&-kashmir',
 'jharkhand',
 'karnataka',
 'kerala',
 'ladakh',
 'lakshadweep',
 'madhya-pradesh',
 'maharashtra',
 'manipur',
 'meghalaya',
 'mizoram',
 'nagaland',
 'odisha',
 'puducherry',
 'punjab',
 'rajasthan',
 'sikkim',
 'tamil-nadu',
 'telangana',
 'tripura',
 'uttar-pradesh',
 'uttarakhand',
 'west-bengal']